#### Install necessary libraries

#### Importing libraries 

In [1]:
import cv2 
import pytesseract
import numpy as np 
import pandas as pd
import re
import gradio as gr
import plotly.express as px
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.cluster import KMeans

#### The Machine Learning model - (Simulated/Neuristic Models)

##### Machine exists No Dataset or csv requred.

## Machine Learning Engines - Syntethic training for zero - dataset Logic

In [2]:
clf = RandomForestClassifier().fit([[0.1], [0.8], [2.1]], [2, 1, 0]) 
reg = RandomForestRegressor().fit([[1.0], [0.0], [-1.0]], [80, 50, 20]) 
km = KMeans(n_clusters=2, n_init=10).fit([[0.5, 0.2], [1.5, 1.8]])

### Image and data processing 

In [3]:
def analyze_game(image):
    try:
        if image is None:
            return "No Image", "N/A", "N/A", "N/A", None, None

        # --- 1. ENHANCED IMAGE PREPROCESSING ---
        image_bgr = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        gray = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2GRAY)
        
        # Scale up significantly (Tesseract loves large, clear numbers)
        gray = cv2.resize(gray, None, fx=3, fy=3, interpolation=cv2.INTER_CUBIC)

        # Denoising and Sharpness
        gray = cv2.GaussianBlur(gray, (1, 1), 0)
        _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        # --- 2. IMPROVED OCR EXTRACTION ---
        # PSM 11 is often better for sparse text like numbers in a sidebar
        custom_config = r'--oem 3 --psm 11 -c tessedit_char_whitelist=0123456789.+- '
        text = pytesseract.image_to_string(thresh, config=custom_config)
        
        # Regex for decimals, negatives, and positives (+1.5, -0.4, 10)
        nums_raw = re.findall(r"[-+]?\d+\.\d+|[-+]?\d+", text)
        nums = [float(n) for n in nums_raw]

        if not nums:
            return "⚠️ No Numbers Detected", "0%", "Check Debug", "Ensure sidebar is visible", None, thresh

        # --- 3. ANALYTICS ---
        df = pd.DataFrame({"Move": range(len(nums)), "Score": nums})
        df['Delta'] = df['Score'].diff().abs().fillna(0)

        # Predict Quality & Probability
        labels = {2: "✅ Good", 1: "⚠️ Mistake", 0: "🚨 Blunder"}
        df['Quality'] = [labels[q] for q in clf.predict(df[['Delta']])]
        df['WinProb'] = reg.predict(df[['Score']])

        # Style Clustering
        style_idx = km.predict([[np.mean(nums), np.std(nums)]])[0]
        playstyle = ["Strategic/Solid", "Aggressive/Tactical"][style_idx]

        # Final Metrics
        accuracy_val = max(0, min(100, 100 - (df['Delta'].mean() * 12)))
        rank = "Elite" if accuracy_val > 85 else "Standard" if accuracy_val > 60 else "Developing"
        pattern = f"Frequent: {df['Quality'].mode()[0]}"

        # DARK THEME PLOT
        fig = px.line(df, x="Move", y="Score", title="Game Advantage (Centipawns)", markers=True)
        fig.add_hline(y=0, line_dash="dash", line_color="red", annotation_text="Equal")
        fig.update_layout(template="plotly_dark", plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)')

        return rank, f"{accuracy_val:.1f}%", playstyle, pattern, fig, thresh

    except Exception as e:
        return f"Error: {str(e)}", "N/A", "N/A", "N/A", None, None

### Gradio Interface 


In [4]:
# --- GRADIO INTERFACE (Same structure, improved labels) ---
with gr.Blocks(theme=gr.themes.Monochrome()) as demo:
    gr.Markdown("# ♟️ Chess ML Vision Analyst")
    gr.Markdown("Upload a screenshot of your **Evaluation/Move sidebar** to analyze.")
    
    with gr.Row():
        input_img = gr.Image(label="Sidebar Screenshot", type="numpy")
        with gr.Column():
            with gr.Row():
                rank_out = gr.Textbox(label="Skill Rank")
                acc_out = gr.Textbox(label="Accuracy %")
            style_out = gr.Textbox(label="Playstyle")
            assoc_out = gr.Textbox(label="Pattern Insight")

    with gr.Tabs():
        with gr.TabItem("📊 Evaluation Graph"):
            plot_out = gr.Plot()
        with gr.TabItem("👁️ Vision Debug"):
            vision_out = gr.Image(label="Processed Binary Image")
    
    btn = gr.Button("Analyze Moves", variant="primary")
    btn.click(analyze_game, input_img, [rank_out, acc_out, style_out, assoc_out, plot_out, vision_out])

demo.launch()

* Running on local URL:  http://127.0.0.1:7860
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://daabc74bed81ddfe66.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


### Computer vision engine 